In [1]:
# Note for instructor/TAs: 
# The two observable trends based on the data are listed in a markdown cell at the bottom of homework code.

In [2]:
# Import dependencies
import pandas as pd

In [3]:
# Load csv files
load_school_data = ("Resources/schools_complete.csv")
load_student_data = ("Resources/students_complete.csv")

In [4]:
# Reading in cvs data
school_data = pd.read_csv(load_school_data)
student_data = pd.read_csv(load_student_data)

# Create school_df to add type of school to final School Summary 
school_df = school_data[["school_name", "type"]]
school_df = school_df.set_index("school_name")


In [5]:
# Merging the 2 dataframes
complete_data = pd.merge(student_data, school_data, how="left", on=["school_name","school_name"])
complete_data.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [6]:
# Creating District Summary

total_schools = complete_data["school_name"].nunique()
total_students = complete_data["student_name"].count()
total_budget = complete_data.drop_duplicates(["school_name"]).groupby("school_name")["budget"].agg("sum").sum().astype("float")
avg_math_score = round(complete_data["math_score"].mean(), 6)
avg_read_score = round(complete_data["reading_score"].mean(), 6)
overall_passing_rate = round((avg_math_score + avg_read_score)/2, 6)
math_pass = round((complete_data["math_score"] >= 70).value_counts().loc[True]/len(complete_data) * 100, 6) 
read_pass = round((complete_data["reading_score"] >= 70).value_counts().loc[True]/len(complete_data) * 100, 6) 

district_summary = pd.DataFrame({"Total Schools": total_schools,
                        "Total Students": total_students,
                        "Total Budget": total_budget,
                        "Avg Math Score": avg_math_score,
                        "Avg Read Score": avg_read_score,
                        "% Passing Math": math_pass,
                        "% Pass Reading": read_pass,
                        "% Overall Pass Rate": [overall_passing_rate]
                        })

district_summary.style.format({"Total Budget": "${:20,.2f}", 
                              "Total Students": "{:,}"
                              })


,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Read Score,% Passing Math,% Pass Reading,% Overall Pass Rate
0,15,"39,170","$ 24,649,428.00",78.9854,81.8778,74.9809,85.8055,80.4316


In [7]:
# Grouping complete_data dataframe by school name
group_school = complete_data.groupby(["school_name"])


In [8]:
# Creating School Summary 

total_students_bs = group_school["student_name"].count()

total_budget_bs = group_school["budget"].unique().astype("float")
per_student_budget_bs = total_budget_bs/total_students_bs

avg_math_score_bs = group_school["math_score"].mean()

avg_read_score_bs = group_school["reading_score"].mean()

math_num_bs = complete_data[complete_data["math_score"] >= 70].groupby(complete_data["school_name"])
pass_math_percent = (math_num_bs["school_name"].count()/total_students_bs)*100

read_num_bs = complete_data[complete_data["reading_score"] >= 70].groupby(complete_data["school_name"])
pass_read_percent = (read_num_bs["school_name"].count()/total_students_bs)*100

overall_pass_percent = (pass_math_percent + pass_read_percent) / 2

school_summary = pd.DataFrame({"Total Student": total_students_bs,
                              "School Type": school_df["type"],
                              "Total School Budget": total_budget_bs,
                              "Per Student Budget": per_student_budget_bs,
                              "Avg Math Score": avg_math_score_bs,
                              "Avg Reading Score": avg_read_score_bs,
                              "% Passing Math": pass_math_percent,
                              "% Passing Reading": pass_read_percent,
                              "Overall Passing Rate": overall_pass_percent
})

school_summary.style.format({"Total School Budget": "${:20,.2f}", 
                             "Per Student Budget": "${:20,.2f}"
                              })


,Total Student,School Type,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,4976,District,"$ 3,124,928.00",$ 628.00,77.0484,81.034,66.6801,81.9333,74.3067
Cabrera High School,1858,Charter,"$ 1,081,356.00",$ 582.00,83.0619,83.9758,94.1335,97.0398,95.5867
Figueroa High School,2949,District,"$ 1,884,411.00",$ 639.00,76.7118,81.158,65.9885,80.7392,73.3639
Ford High School,2739,District,"$ 1,763,916.00",$ 644.00,77.1026,80.7463,68.3096,79.299,73.8043
Griffin High School,1468,Charter,"$ 917,500.00",$ 625.00,83.3515,83.8168,93.3924,97.139,95.2657
Hernandez High School,4635,District,"$ 3,022,020.00",$ 652.00,77.2898,80.9344,66.753,80.863,73.808
Holden High School,427,Charter,"$ 248,087.00",$ 581.00,83.8033,83.815,92.5059,96.2529,94.3794
Huang High School,2917,District,"$ 1,910,635.00",$ 655.00,76.6294,81.1827,65.6839,81.3164,73.5002
Johnson High School,4761,District,"$ 3,094,650.00",$ 650.00,77.0725,80.9664,66.0576,81.2224,73.64
Pena High School,962,Charter,"$ 585,858.00",$ 609.00,83.8399,84.0447,94.5946,95.9459,95.2703


In [11]:
# Finding the Top Performing Schools (By Passing Rate)

top_schools = school_summary.sort_values("Overall Passing Rate", ascending=False)

top_schools.style.format({"Total School Budget": "${:20,.2f}", 
                          "Per Student Budget": "${:20,.2f}"})

top_schools.head(5)


,Total Student,School Type,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,1858,Charter,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,1635,Charter,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,962,Charter,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,1468,Charter,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,2283,Charter,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [12]:
# Finding the Bottom Performing Schools (By Passing Rate)

bottom_schools = school_summary.sort_values("Overall Passing Rate", ascending = True)

bottom_schools.style.format({"Total School Budget": "${:20,.2f}", 
                              "Per Student Budget": "${:20,.2f}"})

bottom_schools.head()

,Total Student,School Type,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,3999,District,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,2949,District,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,2917,District,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,4761,District,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,2739,District,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [23]:
# Finding Math score by Grade

math_df = complete_data[['school_name','grade','math_score']]

# Create panda series for each grade for math

nine_df = math_df.loc[math_df["grade"] == "9th"].groupby(['school_name']).mean()
ten_df = math_df.loc[math_df["grade"] == "10th"].groupby(['school_name']).mean()
eleven_df = math_df.loc[math_df["grade"] == "11th"].groupby(['school_name']).mean()
twelve_df = math_df.loc[math_df["grade"] == "12th"].groupby(['school_name']).mean()

complete_df = pd.merge(nine_df, ten_df, how="outer", on="school_name")
complete_df = complete_df.rename(columns={"math_score_x": "9th",
                                          "math_score_y": "10th"})
complete_df = pd.merge(complete_df, eleven_df, how="outer", on="school_name")
complete_df = complete_df.rename(columns={"math_score": "11th"})
complete_df = pd.merge(complete_df, twelve_df, how="outer", on="school_name")
complete_df = complete_df.rename(columns={"math_score": "12th"})
complete_df

# 2nd way of finding MATH SCORE BY GRADE, but could not sorted correctly
    # math_grade_group = math_df.groupby(['school_name', 'grade']).mean().unstack()
    # math_grade_group.head(15)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [24]:
# Finding Reading Score by Grade
read_df = complete_data[['school_name','grade','reading_score']]

# Create panda series for each grade for reading
nine_df = read_df.loc[read_df["grade"] == "9th"].groupby(['school_name']).mean()
ten_df = read_df.loc[read_df["grade"] == "10th"].groupby(['school_name']).mean()
eleven_df = read_df.loc[read_df["grade"] == "11th"].groupby(['school_name']).mean()
twelve_df = read_df.loc[read_df["grade"] == "12th"].groupby(['school_name']).mean()

complete_df = pd.merge(nine_df, ten_df, how="outer", on="school_name")
complete_df = complete_df.rename(columns={"reading_score_x": "9th",
                                          "reading_score_y": "10th"})
complete_df = pd.merge(complete_df, eleven_df, how="outer", on="school_name")
complete_df = complete_df.rename(columns={"reading_score": "11th"})
complete_df = pd.merge(complete_df, twelve_df, how="outer", on="school_name")
complete_df = complete_df.rename(columns={"reading_score": "12th"})
complete_df.head(15)


# 2nd way of finding READING SCORE BY GRADE, but could not sorted correctly
    # read_grade_group = read_df.groupby(['school_name', 'grade']).mean().unstack()
    # read_grade_group.head(15)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [25]:
# Finding Scores by School spending
school_summary_spend = school_summary[["Per Student Budget","Avg Math Score", "Avg Reading Score", 
                                       "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]

bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_summary_spend["Spending Ranges"] = pd.cut(school_summary_spend["Per Student Budget"], bins, labels=group_names)
school_summary_group_spend = school_summary_spend.groupby("Spending Ranges")
school_summary_group_spend = pd.DataFrame(school_summary_group_spend.mean())
school_summary_group_spend = school_summary_group_spend.drop(axis=1, labels="Per Student Budget")
school_summary_group_spend


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [26]:
# Finding Scores by School Size
school_summary_size = school_summary[["Total Student","Avg Math Score", "Avg Reading Score", 
                                      "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]

bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary_size["School Size"] = pd.cut(school_summary_size["Total Student"], bins, labels=group_names)
school_summary_group_size = school_summary_size.groupby("School Size")
school_summary_group_size = pd.DataFrame(school_summary_group_size.mean())
school_summary_group_size = school_summary_group_size.drop(axis=1, labels="Total Student")
school_summary_group_size


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [27]:
# Finding Scores by School Type
school_summary_type = school_summary[["School Type","Avg Math Score", "Avg Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]
school_summary_group_type = school_summary_type.groupby("School Type")
school_summary_group_type.mean()


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


## Observational Trends


Two observable trends based on the data are:

Trend one: Charter schools tend to have higher percentage passing math and reading scores compared to District schools.

Trend two: Schools with 2000 or less students tend to perform better in math and reading compared to schools with 2000 or more students.